In [ ]:

import os
os.environ["PYTORCH_NVFUSER_DISABLE"] = "1"
os.environ["TORCHDYNAMO_DISABLE"] = "1"


import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [ ]:
!pip install pillow


In [ ]:
from PIL import Image
import numpy as np

def extract_visual_embedding(image_path):
    img = Image.open(image_path).convert("RGB")
    arr = np.array(img)
    h, w, _ = arr.shape

    def crop_percent(x1, y1, x2, y2):
        return arr[int(y1*h):int(y2*h), int(x1*w):int(x2*w), :]

    regions = {
        'eyes': crop_percent(0.3, 0.2, 0.7, 0.35),
        'mouth': crop_percent(0.35, 0.65, 0.65, 0.85),
        'nose': crop_percent(0.45, 0.4, 0.55, 0.6),
        'forehead': crop_percent(0.3, 0.05, 0.7, 0.2),
        'chin': crop_percent(0.4, 0.85, 0.6, 0.95)
    }

    embeddings = {}
    for region_name, region in regions.items():
        flat = region.flatten() / 255.0
        embeddings[region_name] = np.concatenate([
            [np.mean(flat), np.std(flat)],
            np.percentile(flat, [25, 50, 75])
        ])

    final_vector = np.concatenate([v for v in embeddings.values()])
    return final_vector  # Örn: 25 boyutlu vektör


In [ ]:
import random
import json
import numpy as np

def generate_fake_story(vector):
    base_sentences = [
        "Tokat’da doğdum ve büyüdüm.",
        "Hayatım boyunca hep sanatla ilgilendim.",
        "Çocukluğum köyde geçti, çok huzurluydu.",
        "Her zaman yalnız bir insan oldum.",
        "Annem bana umutla yaşamayı öğretti.",
        "Müzik benim en iyi arkadaşım oldu.",
        "Bazen sessizlikte kaybolurum.",
        "Hayat bana sabretmeyi öğretti.",
        "İnsanlara yardım etmek beni mutlu eder.",
        "Duygularımı kelimelere dökmekte zorlanırım."
    ]
    return random.choice(base_sentences)

def generate_dataset(output_file="training_data_1000.json", count=1000):
    with open(output_file, "w", encoding="utf-8") as f:
        for _ in range(count):
            vec = np.random.rand(25).tolist()
            story = generate_fake_story(vec)
            item = {"vector": vec, "text": story}
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
    print(f"✅ Sentetik veri oluşturuldu: {output_file} ({count} adet)")


In [ ]:

import random
import json
import numpy as np
from google.colab import drive


drive.mount('/content/drive')


def generate_fake_story(vector):
    base_sentences = [
        "İstanbul’da doğdum ve büyüdüm.",
        "Hayatım boyunca hep sanatla ilgilendim.",
        "Çocukluğum köyde geçti, çok huzurluydu.",
        "Her zaman yalnız bir insan oldum.",
        "Annem bana umutla yaşamayı öğretti.",
        "Kitaplar benim en iyi arkadaşım oldu.",
        "Bazen sessizlikte kaybolurum.",
        "Hayat bana sabretmeyi öğretti.",
        "İnsanlara yardım etmek beni mutlu eder.",
        "Duygularımı kelimelere dökmekte zorlanırım."
    ]
    return random.choice(base_sentences)


def generate_dataset_and_save(count=5000, file_name="training_data_5000.json"):

    with open(file_name, "w", encoding="utf-8") as f:
        for _ in range(count):
            vec = np.random.rand(25).tolist()
            story = generate_fake_story(vec)
            item = {"vector": vec, "text": story}
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"✅ Sentetik veri oluşturuldu: {file_name} ({count} adet)")


    drive_path = f"/content/drive/MyDrive/{file_name}"
    !cp {file_name} {drive_path}
    print(f"📁 Google Drive'a yüklendi: {drive_path}")


generate_dataset_and_save()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Sentetik veri oluşturuldu: training_data_5000.json (5000 adet)
📁 Google Drive'a yüklendi: /content/drive/MyDrive/training_data_5000.json


In [ ]:

import os
os.environ["PYTORCH_NVFUSER_DISABLE"] = "1"
os.environ["TORCHDYNAMO_DISABLE"] = "1"

import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


EMBED_DIM = 256
FF_DIM = 512
NUM_HEADS = 8
NUM_LAYERS = 4
MAX_LEN = 120               # METİN BOYU DÜŞÜRÜLDÜ
EPOCHS = 50                 # DAHA HAFİF EĞİTİM
BATCH_SIZE = 4              # BELLEK DOSTU
LEARNING_RATE = 5e-4
PATIENCE = 10
MIN_DELTA = 0.02


char_set = list("abcçdefgğıijklmnoöprsştuüvyzABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ0123456789 .,!?-\n")
char2idx = {c: i for i, c in enumerate(char_set)}
idx2char = {i: c for c, i in char2idx.items()}
vocab_size = len(char2idx)

def text_to_tensor(text):
    ids = [char2idx.get(c, 0) for c in text]
    ids += [0] * (MAX_LEN - len(ids))
    return torch.tensor(ids[:MAX_LEN])


class PersonaDataset(Dataset):
    def __init__(self, json_path):
        self.data = []
        with open(json_path, "r", encoding="utf-8") as f:
            for line in f:
                obj = json.loads(line)
                vec = torch.tensor(obj["vector"]).float()
                txt = text_to_tensor(obj["text"])
                self.data.append((vec, txt))
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]


class ScaledDotProductAttention(nn.Module):
    def forward(self, Q, K, V):
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / d_k**0.5
        attn = torch.softmax(scores, dim=-1)
        return torch.matmul(attn, V)

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        self.out = nn.Linear(embed_dim, embed_dim)
        self.attn = ScaledDotProductAttention()
    def forward(self, x):
        B, T, _ = x.size()
        Q = self.q_linear(x).view(B, T, self.num_heads, self.d_k).transpose(1, 2)
        K = self.k_linear(x).view(B, T, self.num_heads, self.d_k).transpose(1, 2)
        V = self.v_linear(x).view(B, T, self.num_heads, self.d_k).transpose(1, 2)
        scores = self.attn(Q, K, V)
        concat = scores.transpose(1, 2).contiguous().view(B, T, self.num_heads * self.d_k)
        return self.out(concat)

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attn = MultiHeadAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim), nn.ReLU(), nn.Linear(ff_dim, embed_dim))
        self.norm2 = nn.LayerNorm(embed_dim)
    def forward(self, x):
        attn = self.attn(x)
        x = self.norm1(x + attn)
        ff = self.ff(x)
        return self.norm2(x + ff)

class EmbeddingLayer(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super().__init__()
        self.projection = nn.Sequential(
            nn.Linear(input_dim, embed_dim), nn.ReLU(), nn.LayerNorm(embed_dim))
    def forward(self, x): return self.projection(x)

class CharDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.layers = nn.ModuleList([TransformerBlock(embed_dim, num_heads, ff_dim) for _ in range(num_layers)])
        self.output = nn.Linear(embed_dim, vocab_size)
    def forward(self, x):
        x = self.embedding(x)
        for layer in self.layers: x = layer(x)
        return self.output(x)

class LifeStoryModel(nn.Module):
    def __init__(self, persona_dim, vocab_size, embed_dim, num_heads, ff_dim, num_layers):
        super().__init__()
        self.embedding_layer = EmbeddingLayer(persona_dim, embed_dim)
        self.decoder = CharDecoder(vocab_size, embed_dim, num_heads, ff_dim, num_layers)
    def forward(self, persona_vec, target_seq):
        context = self.embedding_layer(persona_vec).unsqueeze(1)
        decoder_input = self.decoder.embedding(target_seq) + context
        for layer in self.decoder.layers:
            decoder_input = layer(decoder_input)
        return self.decoder.output(decoder_input)


print("🎯 Hafif eğitim başladı (max_len=120, batch=4, patience=3)")

model = LifeStoryModel(25, vocab_size, EMBED_DIM, NUM_HEADS, FF_DIM, NUM_LAYERS)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

dataset = PersonaDataset("training_data_5000.json")
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

best_loss = float("inf")
patience_counter = 0

for epoch in range(EPOCHS):
    total_loss = 0
    model.train()
    for persona_vec, target_seq in loader:
        optimizer.zero_grad()
        output = model(persona_vec, target_seq[:, :-1])
        loss = criterion(output.reshape(-1, vocab_size), target_seq[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss:.4f}")

    if (epoch + 1) % 10 == 0:
        fname = f"deeppersona_epoch{epoch+1}.pth"
        torch.save(model.state_dict(), fname)
        print(f" Kayıt tamamlandı: {fname}")

    if best_loss - total_loss > MIN_DELTA:
        best_loss = total_loss
        patience_counter = 0
        torch.save(model.state_dict(), "deeppersona_best.pth")
    else:
        patience_counter += 1
        print(f"⏸️  No improvement. Patience: {patience_counter}/{PATIENCE}")
        if patience_counter >= PATIENCE:
            print(f"🛑 Early stopping. Best Loss: {best_loss:.4f}")
            break



🎯 Hafif eğitim başladı (max_len=120, batch=4, patience=3)
Epoch 1/50 | Loss: 518.4734
Epoch 2/50 | Loss: 446.0169
Epoch 3/50 | Loss: 443.7130
Epoch 4/50 | Loss: 460.1929
⏸️  No improvement. Patience: 1/10
Epoch 5/50 | Loss: 442.2466
Epoch 6/50 | Loss: 441.8353
Epoch 7/50 | Loss: 476.0089
⏸️  No improvement. Patience: 1/10
Epoch 8/50 | Loss: 442.1511
⏸️  No improvement. Patience: 2/10
Epoch 9/50 | Loss: 456.8483
⏸️  No improvement. Patience: 3/10
Epoch 10/50 | Loss: 441.3861
💾 Kayıt tamamlandı: deeppersona_epoch10.pth
Epoch 11/50 | Loss: 441.2829
Epoch 12/50 | Loss: 587.9491
⏸️  No improvement. Patience: 1/10
Epoch 13/50 | Loss: 442.7393
⏸️  No improvement. Patience: 2/10
Epoch 14/50 | Loss: 534.1420
⏸️  No improvement. Patience: 3/10
Epoch 15/50 | Loss: 468.7375
⏸️  No improvement. Patience: 4/10
Epoch 16/50 | Loss: 441.1829
Epoch 17/50 | Loss: 584.4429
⏸️  No improvement. Patience: 1/10
Epoch 18/50 | Loss: 442.6785
⏸️  No improvement. Patience: 2/10
Epoch 19/50 | Loss: 616.8066
⏸️  No

In [ ]:
model.load_state_dict(torch.load("deeppersona_best.pth"))
model.eval()


LifeStoryModel(
  (embedding_layer): EmbeddingLayer(
    (projection): Sequential(
      (0): Linear(in_features=25, out_features=256, bias=True)
      (1): ReLU()
      (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (decoder): CharDecoder(
    (embedding): Embedding(74, 256)
    (layers): ModuleList(
      (0-3): 4 x TransformerBlock(
        (attn): MultiHeadAttention(
          (q_linear): Linear(in_features=256, out_features=256, bias=True)
          (k_linear): Linear(in_features=256, out_features=256, bias=True)
          (v_linear): Linear(in_features=256, out_features=256, bias=True)
          (out): Linear(in_features=256, out_features=256, bias=True)
          (attn): ScaledDotProductAttention()
        )
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff): Sequential(
          (0): Linear(in_features=256, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=256, bias=Tr

In [ ]:
def generate_text(model, persona_vec, char2idx, idx2char, max_len=120, temperature=0.9):
    model.eval()
    input_ids = [char2idx.get("B", 0)]
    for _ in range(max_len):
        seq_tensor = torch.tensor(input_ids).unsqueeze(0)
        with torch.no_grad():
            logits = model(persona_vec.unsqueeze(0), seq_tensor)[0, -1]
            probs = F.softmax(logits / temperature, dim=0).cpu().numpy()
            next_id = np.random.choice(len(probs), p=probs)
        input_ids.append(next_id)
        if idx2char[next_id] == "\n":
            break
    return "".join([idx2char[i] for i in input_ids])


In [ ]:
!pip install gradio Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00


In [1]:
!pip install gradio
import gradio as gr
from PIL import Image
import torch
import numpy as np



def extract_visual_embedding(image: Image.Image):
    arr = np.array(image.resize((224, 224))).astype(np.float32) / 255.0
    features = [
        np.mean(arr), np.std(arr), np.min(arr), np.max(arr),
        *np.percentile(arr, [25, 50, 75])
    ]
    return torch.tensor(features + [0]*(25-len(features)), dtype=torch.float32)  # 25 boyut sabitlenmiş


def generate_text(model, persona_vec, char2idx, idx2char, max_len=300, temperature=1.1):
    model.eval()
    input_ids = [char2idx.get("B", 0)]  # Başlangıç karakteri
    for _ in range(max_len):
        seq_tensor = torch.tensor(input_ids).unsqueeze(0)
        with torch.no_grad():
            logits = model(persona_vec.unsqueeze(0), seq_tensor)[0, -1]
            probs = torch.softmax(logits / temperature, dim=0).cpu().numpy()
            next_id = np.random.choice(len(probs), p=probs)
        input_ids.append(next_id)
        if idx2char[next_id] == "\n":
            break
    return "".join([idx2char[i] for i in input_ids])


ModuleNotFoundError: No module named 'gradio'

In [ ]:
def pipeline(image):
    vec = extract_visual_embedding(image)
    story = generate_text(model, vec, char2idx, idx2char)
    return story

demo = gr.Interface(
    fn=pipeline,
    inputs=gr.Image(type="pil", label="Bir yüz fotoğrafı yükleyin"),
    outputs=gr.Textbox(label="DeepPersona'nın yazdığı hayat hikayesi"),
    title="🧠 BMinator – AI Hikaye Üretici",
    description="Yüz görüntüsüne dayalı olarak hayat hikayesi yazan üretken yapay zeka sistemine hoşgeldiniz sonsuz hayal gücünün tadını çıkarın.",
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1039420fb2cdacb01d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
